In [1]:
import pandas as pd
import wandb
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from src.utils import *

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: minjabenho. Use `wandb login --relogin` to force relogin


True

In [3]:
# Datasets
BASELINE_TRAIN = "data/train/baseline_train.csv"
BASELINE_VAL = "data/train/baseline_val.csv"

TRUNCATED_BASELINE_TRAIN = "data/train/baseline-truncated_train.csv"
TRUNCATED_BASLINE_VAL = "data/train/baseline-truncated_val.csv"

BASELINE_W_FEAT_ENG_TRAIN = "data/train/baseline-w-feature-eng_train.csv"
BASELINE_W_FEAT_ENG_VAL = "data/train/baseline-w-feature-eng_val.csv"

TRUNCATED_FEAT_ENG_TRAIN = "data/train/truncated-feat-eng_train.csv"
TRUNCATED_FEAT_ENG_VAL = "data/train/truncated-feat-eng_val.csv"

# Note: this dataset has data leakage and should not be used for training
TARGET_ENC_BASELINE_TRAIN = "data/train/target_encoding_baseline_train.csv"
TARGET_ENC_BASELINE_VAL = "data/train/target_encoding_baseline_val.csv"

### Model For the Baseline Dataset

In [ ]:
sweep_config = {
    'method': 'bayes'
}
metric = {
    'name': 'val_rmse',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric


In [ ]:
parameters_dict = {
    'learning_rate': {
        'values': [0.1]
    },
    'max_iter': {
        'values': [100, 200, 300, 500, 1000, 2000]
    },
    'max_leaf_nodes': {
        'values': [10, 31, 50, 100]
    },
    'max_depth' : {
        'values': [3, 4, 5, 6, 7, 8, 9]
    },
    'min_samples_leaf': {
        'values': [20, 40, 60]
    },
    'l2_regularization': {
        'values': [0, 0.1, 0.2, 0.3]
    },
    'max_bins': {
        'values': [255, 128]
    }
}
sweep_config['parameters'] = parameters_dict

In [ ]:
TRAIN_DATA = BASELINE_TRAIN
VAL_DATA = BASELINE_VAL

In [ ]:
train_set = pd.read_csv(TRAIN_DATA)
val_set = pd.read_csv(VAL_DATA)

X_train, y_train = split_features_and_monthly_rent_label(train_set)
X_test, y_test = split_features_and_monthly_rent_label(val_set)

In [ ]:
def train():
    with wandb.init():
        config = wandb.config

        regressor = HistGradientBoostingRegressor(
            random_state=42,
            learning_rate=config.learning_rate,
            max_iter=config.max_iter,
            max_depth=config.max_depth,
            max_leaf_nodes=config.max_leaf_nodes,
            min_samples_leaf=config.min_samples_leaf,
            l2_regularization=config.l2_regularization,
            max_bins=config.max_bins)
        regressor.fit(X_train, y_train)
        train_score = regressor.score(X_train, y_train)
        val_score = regressor.score(X_test, y_test)
        
        y_train_pred = regressor.predict(X_train)
        train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
        y_test_pred = regressor.predict(X_test)
        val_rmse = mean_squared_error(y_test, y_test_pred, squared=False)


    
        # feat_importances = pd.Series(regressor.feature_importances_, index=X_train.columns)
        # plt.barh(X_train.columns, feat_importances.nlargest(10))
        # wandb.log({'feature_importances': plt})

        # wandb.sklearn.plot_regressor(regressor, X_train, X_test, y_train, y_test, 'GradientBoost')
        wandb.log(data={"train_rmse": train_rmse, "train_score": train_score, "val_rmse": val_rmse, "val_score": val_score})



In [ ]:
sweep_id = wandb.sweep(sweep_config, project="hdb_rental_prices_baseline_rmse_histgb")
wandb.agent(sweep_id, train, count=20)

### Model for the Truncated Baseline Set

In [ ]:
sweep_config = {
    'method': 'bayes'
}
metric = {
    'name': 'mse',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [ ]:
parameters_dict = {
    'learning_rate': {
        'values': [0.1]
    },
    'max_iter': {
        'values': [100, 200, 300, 500, 1000, 2000]
    },
    'max_leaf_nodes': {
        'values': [10, 31, 50, 100]
    },
    'max_depth' : {
        'values': [3, 4, 5, 6, 7, 8, 9]
    },
    'min_samples_leaf': {
        'values': [20, 40, 60]
    },
    'l2_regularization': {
        'values': [0, 0.1, 0.2, 0.3]
    },
    'max_bins': {
        'values': [255, 128]
    }
}
sweep_config['parameters'] = parameters_dict

In [ ]:
TRAIN_DATA = TRUNCATED_BASELINE_TRAIN
VAL_DATA = TRUNCATED_BASLINE_VAL

In [ ]:
train_set = pd.read_csv(TRAIN_DATA)
val_set = pd.read_csv(VAL_DATA)

X_train, y_train = split_features_and_monthly_rent_label(train_set)
X_test, y_test = split_features_and_monthly_rent_label(val_set)

In [ ]:
def train():
    with wandb.init():
        config = wandb.config

        regressor = HistGradientBoostingRegressor(
            random_state=42,
            learning_rate=config.learning_rate,
            n_estimators=config.n_estimators,
            max_depth=config.max_depth,
            subsample=config.subsample,
            min_samples_leaf=config.min_samples_leaf,
            min_samples_split=config.min_samples_split,
            min_impurity_decrease=config.min_impurity_decrease,
            min_weight_fraction_leaf=config.min_weight_fraction_leaf)
        regressor.fit(X_train, y_train)
        score = regressor.score(X_test, y_test)
        y_pred = regressor.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

    
        # feat_importances = pd.Series(regressor.feature_importances_, index=X_train.columns)
        # plt.barh(X_train.columns, feat_importances.nlargest(10))
        # wandb.log({'feature_importances': plt})

        # wandb.sklearn.plot_regressor(regressor, X_train, X_test, y_train, y_test, 'GradientBoost')
        wandb.log(data={"mse": mse, "coefficient_of_determination": score})

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="hdb_rental_prices_truncated_baseline")
wandb.agent(sweep_id, train, count=20)

### Model for the Feature Engineering Set

In [ ]:
sweep_config = {
    'method': 'bayes'
}
metric = {
    'name': 'mse',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [ ]:
parameters_dict = {
    'learning_rate': {
        'values': [0.1]
    },
    'max_iter': {
        'values': [100, 200, 300, 500, 1000, 2000]
    },
    'max_leaf_nodes': {
        'values': [10, 31, 50, 100]
    },
    'max_depth' : {
        'values': [3, 4, 5, 6, 7, 8, 9]
    },
    'min_samples_leaf': {
        'values': [20, 40, 60]
    },
    'l2_regularization': {
        'values': [0, 0.1, 0.2, 0.3]
    },
    'max_bins': {
        'values': [255, 128]
    }
}
sweep_config['parameters'] = parameters_dict

In [ ]:
TRAIN_DATA = BASELINE_W_FEAT_ENG_TRAIN
VAL_DATA = BASELINE_W_FEAT_ENG_VAL

In [ ]:
train_set = pd.read_csv(TRAIN_DATA)
val_set = pd.read_csv(VAL_DATA)

X_train, y_train = split_features_and_monthly_rent_label(train_set)
X_test, y_test = split_features_and_monthly_rent_label(val_set)

In [ ]:
def train():
    with wandb.init():
        config = wandb.config

        regressor = HistGradientBoostingRegressor(
            random_state=42,
            learning_rate=config.learning_rate,
            max_iter=config.max_iter,
            max_depth=config.max_depth,
            max_leaf_nodes=config.max_leaf_nodes,
            min_samples_leaf=config.min_samples_leaf,
            l2_regularization=config.l2_regularization,
            max_bins=config.max_bins)
        regressor.fit(X_train, y_train)
        train_score = regressor.score(X_train, y_train)
        val_score = regressor.score(X_test, y_test)
        
        y_train_pred = regressor.predict(X_train)
        train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
        y_test_pred = regressor.predict(X_test)
        val_rmse = mean_squared_error(y_test, y_test_pred, squared=False)


    
        # feat_importances = pd.Series(regressor.feature_importances_, index=X_train.columns)
        # plt.barh(X_train.columns, feat_importances.nlargest(10))
        # wandb.log({'feature_importances': plt})

        # wandb.sklearn.plot_regressor(regressor, X_train, X_test, y_train, y_test, 'GradientBoost')
        wandb.log(data={"train_rmse": train_rmse, "train_score": train_score, "val_rmse": val_rmse, "val_score": val_score})

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="hdb_rental_prices_feature_eng_rmse_histgb")
wandb.agent(sweep_id, train, count=20)

### Model for the Price Per Square Metre Set

In [ ]:
sweep_config = {
    'method': 'bayes'
}
metric = {
    'name': 'val_rmse',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [ ]:
parameters_dict = {
    'learning_rate': {
        'values': [0.1]
    },
    'max_iter': {
        'values': [100, 200, 300, 500, 1000, 2000]
    },
    'max_leaf_nodes': {
        'values': [10, 31, 50, 100]
    },
    'max_depth' : {
        'values': [3, 4, 5, 6, 7, 8, 9]
    },
    'min_samples_leaf': {
        'values': [20, 40, 60]
    },
    'l2_regularization': {
        'values': [0, 0.1, 0.2, 0.3]
    },
    'max_bins': {
        'values': [255, 128]
    }
}
sweep_config['parameters'] = parameters_dict

In [ ]:
TRAIN_DATA = BASELINE_TRAIN
VAL_DATA = BASELINE_VAL

In [ ]:
train_set = pd.read_csv(TRAIN_DATA)
val_set = pd.read_csv(VAL_DATA)

X_train, y_train, _ = split_features_and_rent_per_sqm_label(train_set)
X_test, y_test, _ = split_features_and_rent_per_sqm_label(val_set)

In [ ]:
def train():
    with wandb.init():
        config = wandb.config

        regressor = HistGradientBoostingRegressor(
            random_state=42,
            learning_rate=config.learning_rate,
            max_iter=config.max_iter,
            max_depth=config.max_depth,
            max_leaf_nodes=config.max_leaf_nodes,
            min_samples_leaf=config.min_samples_leaf,
            l2_regularization=config.l2_regularization,
            max_bins=config.max_bins)
        regressor.fit(X_train, y_train)
        train_score = regressor.score(X_train, y_train)
        val_score = regressor.score(X_test, y_test)
        
        y_train_pred = regressor.predict(X_train)
        train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
        y_test_pred = regressor.predict(X_test)
        val_rmse = mean_squared_error(y_test, y_test_pred, squared=False)


    
        # feat_importances = pd.Series(regressor.feature_importances_, index=X_train.columns)
        # plt.barh(X_train.columns, feat_importances.nlargest(10))
        # wandb.log({'feature_importances': plt})

        # wandb.sklearn.plot_regressor(regressor, X_train, X_test, y_train, y_test, 'GradientBoost')
        wandb.log(data={"train_rmse": train_rmse, "train_score": train_score, "val_rmse": val_rmse, "val_score": val_score})

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="hdb_rental_prices_ppsm_baseline_rmse_histgb")
wandb.agent(sweep_id, train, count=20)

### Model for the Price Per Square Metre Set with Feature Engineering

In [ ]:
sweep_config = {
    'method': 'bayes'
}
metric = {
    'name': 'mse',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [ ]:
parameters_dict = {
    'learning_rate': {
        'values': [0.01, 0.1]
    },
    'n_estimators': {
        'values': [100, 200, 300, 500]
    },
    'min_samples_split': {
        'values': [4, 16, 32]
    },
    'min_samples_leaf': {
        'values': [10, 20, 30]
    },
    'max_depth': {
        'values': [3, 6, 9]
    },
    'min_impurity_decrease': {
        'values': [0.1, 0.2]
    },
    'min_weight_fraction_leaf': {
        'values': [0.0, 0.1]
    },
    'subsample': {
        'values': [0.75, 0.5]
    }
}
sweep_config['parameters'] = parameters_dict

In [ ]:
TRAIN_DATA = BASELINE_W_FEAT_ENG_TRAIN
VAL_DATA = BASELINE_W_FEAT_ENG_VAL

In [ ]:
train_set = pd.read_csv(TRAIN_DATA)
val_set = pd.read_csv(VAL_DATA)

X_train, y_train, _ = split_features_and_rent_per_sqm_label(train_set)
X_test, y_test, _ = split_features_and_rent_per_sqm_label(val_set)

In [ ]:
def train():
    with wandb.init():
        config = wandb.config

        regressor = HistGradientBoostingRegressor(
            random_state=42,
            learning_rate=config.learning_rate,
            n_estimators=config.n_estimators,
            max_depth=config.max_depth,
            subsample=config.subsample,
            min_samples_leaf=config.min_samples_leaf,
            min_samples_split=config.min_samples_split,
            min_impurity_decrease=config.min_impurity_decrease,
            min_weight_fraction_leaf=config.min_weight_fraction_leaf)
        regressor.fit(X_train, y_train)
        
        score = regressor.score(X_test, y_test)
        y_pred = regressor.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

    
        # feat_importances = pd.Series(regressor.feature_importances_, index=X_train.columns)
        # plt.barh(X_train.columns, feat_importances.nlargest(10))
        # wandb.log({'feature_importances': plt})

        # wandb.sklearn.plot_regressor(regressor, X_train, X_test, y_train, y_test, 'GradientBoost')
        wandb.log(data={"mse": mse, "coefficient_of_determination": score})

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="hdb_rental_prices_ppsm_feat_eng")
wandb.agent(sweep_id, train, count=20)

### Model for Baseline Target Encoding Dataset

In [ ]:
sweep_config = {
    'method': 'random'
}
metric = {
    'name': 'val_rmse',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [ ]:
parameters_dict = {
    'learning_rate': {
        'values': [0.1]
    },
    'max_iter': {
        'values': [100, 200, 300, 500, 1000, 2000]
    },
    'max_leaf_nodes': {
        'values': [10, 31, 50, 100]
    },
    'max_depth' : {
        'values': [3, 4, 5, 6, 7, 8, 9]
    },
    'min_samples_leaf': {
        'values': [20, 40, 60]
    },
    'l2_regularization': {
        'values': [0, 0.1, 0.2, 0.3]
    },
    'max_bins': {
        'values': [255, 128]
    }
}
sweep_config['parameters'] = parameters_dict

In [ ]:
TRAIN_DATA = TARGET_ENC_BASELINE_TRAIN
VAL_DATA = TARGET_ENC_BASELINE_VAL

In [ ]:
train_set = pd.read_csv(TRAIN_DATA)
val_set = pd.read_csv(VAL_DATA)

X_train, y_train = split_features_and_monthly_rent_label(train_set)
X_test, y_test = split_features_and_monthly_rent_label(val_set)


In [ ]:
def train():
    with wandb.init():
        config = wandb.config

        regressor = HistGradientBoostingRegressor(
            random_state=42,
            learning_rate=config.learning_rate,
            max_iter=config.max_iter,
            max_depth=config.max_depth,
            max_leaf_nodes=config.max_leaf_nodes,
            min_samples_leaf=config.min_samples_leaf,
            l2_regularization=config.l2_regularization,
            max_bins=config.max_bins)
        regressor.fit(X_train, y_train)
        train_score = regressor.score(X_train, y_train)
        val_score = regressor.score(X_test, y_test)
        
        y_train_pred = regressor.predict(X_train)
        train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
        y_test_pred = regressor.predict(X_test)
        val_rmse = mean_squared_error(y_test, y_test_pred, squared=False)


    
        # feat_importances = pd.Series(regressor.feature_importances_, index=X_train.columns)
        # plt.barh(X_train.columns, feat_importances.nlargest(10))
        # wandb.log({'feature_importances': plt})

        # wandb.sklearn.plot_regressor(regressor, X_train, X_test, y_train, y_test, 'GradientBoost')
        wandb.log(data={"train_rmse": train_rmse, "train_score": train_score, "val_rmse": val_rmse, "val_score": val_score})


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="hdb_rental_prices_target_enc_baseline_rmse_histgb")
wandb.agent(sweep_id, train, count=20)

### Model for Truncated Baseline + Feature Engineering

In [ ]:
sweep_config = {
    'method': 'bayes'
}
metric = {
    'name': 'val_rmse',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [ ]:
parameters_dict = {
    'learning_rate': {
        'values': [0.1]
    },
    'max_iter': {
        'values': [100, 200, 300, 500, 1000, 2000]
    },
    'max_leaf_nodes': {
        'values': [10, 31, 50, 100]
    },
    'max_depth' : {
        'values': [3, 4, 5, 6, 7, 8, 9]
    },
    'min_samples_leaf': {
        'values': [20, 40, 60]
    },
    'l2_regularization': {
        'values': [0, 0.1, 0.2, 0.3]
    },
    'max_bins': {
        'values': [255, 128]
    }
}
sweep_config['parameters'] = parameters_dict

In [ ]:
TRAIN_DATA = TRUNCATED_FEAT_ENG_TRAIN
VAL_DATA = TRUNCATED_FEAT_ENG_VAL

In [ ]:
train_set = pd.read_csv(TRAIN_DATA)
val_set = pd.read_csv(VAL_DATA)

X_train, y_train = split_features_and_monthly_rent_label(train_set)
X_test, y_test = split_features_and_monthly_rent_label(val_set)

In [ ]:
def train():
    with wandb.init():
        config = wandb.config

        regressor = HistGradientBoostingRegressor(
            random_state=42,
            learning_rate=config.learning_rate,
            max_iter=config.max_iter,
            max_depth=config.max_depth,
            max_leaf_nodes=config.max_leaf_nodes,
            min_samples_leaf=config.min_samples_leaf,
            l2_regularization=config.l2_regularization,
            max_bins=config.max_bins)
        regressor.fit(X_train, y_train)
        train_score = regressor.score(X_train, y_train)
        val_score = regressor.score(X_test, y_test)
        
        y_train_pred = regressor.predict(X_train)
        train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
        y_test_pred = regressor.predict(X_test)
        val_rmse = mean_squared_error(y_test, y_test_pred, squared=False)


    
        # feat_importances = pd.Series(regressor.feature_importances_, index=X_train.columns)
        # plt.barh(X_train.columns, feat_importances.nlargest(10))
        # wandb.log({'feature_importances': plt})

        # wandb.sklearn.plot_regressor(regressor, X_train, X_test, y_train, y_test, 'GradientBoost')
        wandb.log(data={"train_rmse": train_rmse, "train_score": train_score, "val_rmse": val_rmse, "val_score": val_score})


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="hdb_rental_prices_truncated_feat_eng_rmse_histgb")
wandb.agent(sweep_id, train, count=20)

### Model for the Truncated Model + Feature Engineering in Price Per Sq Metre

In [4]:
sweep_config = {
    'method': 'bayes'
}
metric = {
    'name': 'val_rmse',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [5]:
parameters_dict = {
    'learning_rate': {
        'values': [0.1]
    },
    'max_iter': {
        'values': [100, 200, 300, 500, 1000, 2000]
    },
    'max_leaf_nodes': {
        'values': [10, 31, 50, 100]
    },
    'max_depth' : {
        'values': [3, 4, 5, 6, 7, 8, 9]
    },
    'min_samples_leaf': {
        'values': [20, 40, 60]
    },
    'l2_regularization': {
        'values': [0, 0.1, 0.2, 0.3]
    },
    'max_bins': {
        'values': [255, 128]
    }
}
sweep_config['parameters'] = parameters_dict

In [6]:
TRAIN_DATA = TRUNCATED_FEAT_ENG_TRAIN
VAL_DATA = TRUNCATED_FEAT_ENG_VAL

In [7]:
train_set = pd.read_csv(TRAIN_DATA)
val_set = pd.read_csv(VAL_DATA)

X_train, y_train, _ = split_features_and_rent_per_sqm_label(train_set)
X_test, y_test, _ = split_features_and_rent_per_sqm_label(val_set)

In [8]:
def train():
    with wandb.init():
        config = wandb.config

        regressor = HistGradientBoostingRegressor(
            random_state=42,
            learning_rate=config.learning_rate,
            max_iter=config.max_iter,
            max_depth=config.max_depth,
            max_leaf_nodes=config.max_leaf_nodes,
            min_samples_leaf=config.min_samples_leaf,
            l2_regularization=config.l2_regularization,
            max_bins=config.max_bins)
        regressor.fit(X_train, y_train)
        train_score = regressor.score(X_train, y_train)
        val_score = regressor.score(X_test, y_test)
        
        y_train_pred = regressor.predict(X_train)
        train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
        y_test_pred = regressor.predict(X_test)
        val_rmse = mean_squared_error(y_test, y_test_pred, squared=False)


    
        # feat_importances = pd.Series(regressor.feature_importances_, index=X_train.columns)
        # plt.barh(X_train.columns, feat_importances.nlargest(10))
        # wandb.log({'feature_importances': plt})

        # wandb.sklearn.plot_regressor(regressor, X_train, X_test, y_train, y_test, 'GradientBoost')
        wandb.log(data={"train_rmse": train_rmse, "train_score": train_score, "val_rmse": val_rmse, "val_score": val_score})

In [9]:
sweep_id = wandb.sweep(sweep_config, project="hdb_rental_prices_truncated_feat_eng_ppsm_rmse_histgb")
wandb.agent(sweep_id, train, count=20)

Create sweep with ID: jjwi9k0g
Sweep URL: https://wandb.ai/minjabenho/hdb_rental_prices_truncated_feat_eng_ppsm_rmse_histgb/sweeps/jjwi9k0g


wandb: Agent Starting Run: l5affdxe with config:
wandb: 	l2_regularization: 0
wandb: 	learning_rate: 0.1
wandb: 	max_bins: 128
wandb: 	max_depth: 3
wandb: 	max_iter: 100
wandb: 	max_leaf_nodes: 50
wandb: 	min_samples_leaf: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


train_rmse,▁
train_score,▁
val_rmse,▁
val_score,▁
train_rmse,5.73731
train_score,0.59848
val_rmse,5.74392
val_score,0.58752


wandb: Agent Starting Run: h632433j with config:
wandb: 	l2_regularization: 0.2
wandb: 	learning_rate: 0.1
wandb: 	max_bins: 255
wandb: 	max_depth: 8
wandb: 	max_iter: 1000
wandb: 	max_leaf_nodes: 50
wandb: 	min_samples_leaf: 40
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


train_rmse,▁
train_score,▁
val_rmse,▁
val_score,▁
train_rmse,5.31205
train_score,0.6558
val_rmse,5.59716
val_score,0.60833


...

train_rmse,▁
train_score,▁
val_rmse,▁
val_score,▁
train_rmse,5.38656
train_score,0.64607
val_rmse,5.60197
val_score,0.60765


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 1q1k9z16 with config:
wandb: 	l2_regularization: 0.3
wandb: 	learning_rate: 0.1
wandb: 	max_bins: 255
wandb: 	max_depth: 9
wandb: 	max_iter: 100
wandb: 	max_leaf_nodes: 31
wandb: 	min_samples_leaf: 60
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


train_rmse,▁
train_score,▁
val_rmse,▁
val_score,▁
train_rmse,5.51526
train_score,0.62896
val_rmse,5.63219
val_score,0.60341


wandb: Agent Starting Run: yp0i8o4q with config:
wandb: 	l2_regularization: 0.2
wandb: 	learning_rate: 0.1
wandb: 	max_bins: 255
wandb: 	max_depth: 9
wandb: 	max_iter: 2000
wandb: 	max_leaf_nodes: 100
wandb: 	min_samples_leaf: 60
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


train_rmse,▁
train_score,▁
val_rmse,▁
val_score,▁
train_rmse,5.2749
train_score,0.66059
val_rmse,5.62542
val_score,0.60436
